Fonction qui permet de prendre des photos avec un délais en secondes cf doc : https://docs.python.org/3/library/time.html#time.sleep

In [ ]:
import time
import picamera

def capture_image(output_file):
    with picamera.PiCamera() as camera:
        # Optional: Set camera resolution
        # camera.resolution = (1280, 720)
        
        # Optional: Set camera rotation (0, 90, 180, or 270 degrees)
        # camera.rotation = 90
        
        # Capture an image
        camera.start_preview()
        # Add a delay if needed to let the camera adjust to lighting conditions
        time.sleep(2)
        camera.capture(output_file)
        camera.stop_preview()

if __name__ == "__main__":
    output_file = "image.jpg"  # Change this to the desired output filename
    capture_image(output_file)
    print(f"Image captured and saved as {output_file}")


La même chose mais pour une vidéo

In [ ]:
import picamera
from time import sleep

def capture_video(output_file, duration):
    with picamera.PiCamera() as camera:
        # Optional: Set camera resolution
        # camera.resolution = (1280, 720)
        
        # Optional: Set camera rotation (0, 90, 180, or 270 degrees)
        # camera.rotation = 90
        
        camera.start_recording(output_file)
        sleep(duration)
        camera.stop_recording()

if __name__ == "__main__":
    output_file = "video.h264"  # Change this to the desired output filename
    duration = 10  # Change this to the desired duration of the video in seconds
    capture_video(output_file, duration)
    print(f"Video captured and saved as {output_file}")


Un premier système d'annotation utilisant yolov3

In [ ]:
import cv2
import numpy as np
import picamera
import picamera.array

def track_objects(camera_resolution=(640, 480)):
    # Charger le modèle de détection d'objets (exemple : YOLOv3)
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # Initialiser le suivi de Centroid
    trackers = cv2.MultiTracker_create()

    with picamera.PiCamera() as camera:
        camera.resolution = camera_resolution
        with picamera.array.PiRGBArray(camera) as stream:
            for frame in camera.capture_continuous(stream, format='bgr', use_video_port=True):
                img = frame.array

                if len(trackers.getObjects()) == 0:
                    # Pas de suivi en cours, détecter les objets
                    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
                    net.setInput(blob)
                    outs = net.forward(output_layers)

                    # Traiter les détections
                    boxes = []
                    for out in outs:
                        for detection in out:
                            scores = detection[5:]
                            class_id = np.argmax(scores)
                            confidence = scores[class_id]
                            if confidence > 0.5:
                                # Récupérer les coordonnées de la boîte englobante
                                center_x = int(detection[0] * camera_resolution[0])
                                center_y = int(detection[1] * camera_resolution[1])
                                w = int(detection[2] * camera_resolution[0])
                                h = int(detection[3] * camera_resolution[1])

                                boxes.append((center_x - w // 2, center_y - h // 2, w, h))

                    # Initialiser les trackers pour les boîtes englobantes détectées
                    for box in boxes:
                        tracker = cv2.TrackerCSRT_create()
                        trackers.add(tracker, img, (box[0], box[1], box[2], box[3]))

                # Actualiser le suivi pour chaque objet
                (success, boxes) = trackers.update(img)
                for box in boxes:
                    (x, y, w, h) = [int(v) for v in box]
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # Afficher le flux vidéo avec les boîtes englobantes
                cv2.imshow('Camera', img)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                stream.truncate(0)

    cv2.destroyAllWindows()

if __name__ == "__main__":
    track_objects()


Dans cet exemple :

    Nous avons utilisé le suivi de Centroid pour suivre les objets détectés dans le flux vidéo.
    Lorsqu'aucun objet n'est suivi, nous détectons les objets à l'aide de YOLOv3 et initialisons les trackers pour ces objets.
    Les trackers sont ensuite mis à jour à chaque image du flux vidéo pour suivre les objets détectés.

Assurez-vous d'avoir téléchargé les fichiers du modèle YOLOv3 (yolov3.weights, yolov3.cfg, coco.names) et d'avoir les bibliothèques OpenCV et picamera installées sur votre Raspberry Pi. Vous devrez peut-être ajuster les chemins des fichiers du modèle et les paramètres de résolution de la caméra en fonction de vos besoins.